# **Анализ A/B тестирования**

In [51]:
# Импорт необходимых библиотек
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

## **1. Предварительный анализ данных**
### 1.1. Загрузка и агрегация данных

In [54]:
# Загрузка данных
data = pd.read_csv('synthetic_gmv_data_1.1.csv')
print(f"Размерность данных: {data.shape}")
print("\nПервые 5 строк данных:")
print(data.head())

# Агрегация данных по пользователям
user_gmv = data.groupby('user_id')['gmv'].sum().reset_index()

Размерность данных: (1601993, 2)

Первые 5 строк данных:
      user_id  gmv
0  guo0pdwqvg  506
1  guo0pdwqvg  760
2  guo0pdwqvg  633
3  guo0pdwqvg  506
4  guo0pdwqvg  760


### 1.2. Расчет основных статистик

In [55]:
# Расчет выборочного среднего и несмещенной дисперсии
mean_gmv = user_gmv['gmv'].mean()
variance_gmv = user_gmv['gmv'].var(ddof=1)

print(f"Оценка математического ожидания GMV: {mean_gmv:.2f}")
print(f"Несмещенная оценка дисперсии GMV: {variance_gmv:.2f}")

Оценка математического ожидания GMV: 2750.85
Несмещенная оценка дисперсии GMV: 3646228.41


## **2. Расчет размера выборки для A/B теста**

Расчитаем необходимое количество пользователей, чтобы "увидеть" эффект в 1% с вероятностью ошибки первого рода 5% и вероятностью ошибки второго рода 20%. Разбиение на тестовую и контрольную группу 1 к 3 (контроль в 3 раза больше теста). Дисперсии в тестовой и контрольной группах одинаковые.

In [58]:
def calculate_sample_size(variance, effect, k, alpha=0.05, beta=0.2):
    """
    Расчет необходимого размера выборки для обнаружения эффекта

    Параметры:
    variance - дисперсия метрики
    effect - минимальный обнаруживаемый эффект (в абсолютных единицах)
    k - соотношение контрольной группы к тестовой (k:1)
    alpha - вероятность ошибки первого рода
    beta - вероятность ошибки второго рода

    Возвращает:
     размер тестовой группы, размер контрольной группы, общий размер
    """
    dist = stats.norm(loc=0, scale=1)
    const = (dist.ppf(1-alpha/2) + dist.ppf(1-beta))**2
    m = const * (1 + k) * variance / (effect**2)
    n = m / k
    return int(n), int(m), int(n + m)

# Параметры теста
alpha = 0.05  # вероятность ошибки первого рода
beta = 0.2    # вероятность ошибки второго рода
effect = 0.01 * mean_gmv  # 1% эффект от среднего
k = 3         # соотношение контрольной к тестовой группе 3:1

# Расчет размера выборки
test_size, control_size, total_size = calculate_sample_size(
    variance=variance_gmv,
    effect=effect,
    k=k,
    alpha=alpha,
    beta=beta
)

print(f"- Тестовая группа: {test_size} пользователей")
print(f"- Контрольная группа: {control_size} пользователей")
print(f"- Всего: {total_size} пользователей")

- Тестовая группа: 50426 пользователей
- Контрольная группа: 151279 пользователей
- Всего: 201705 пользователей


## **3. Анализ A/B теста: сравнение средних GMV**
3.1. Загрузка и подготовка данных

In [59]:
# Загрузка данных эксперимента
data_exp = pd.read_csv('synthetic_gmv_data_1.2.csv')
print(f"Размерность данных эксперимента: {data_exp.shape}")
print("\nПервые 5 строк данных эксперимента:")
print(data_exp.head())

# Агрегация по пользователям
user_gmv_exp = data_exp.groupby(['user_id', 'group_name'])['gmv'].sum().reset_index()


Размерность данных эксперимента: (799061, 3)

Первые 5 строк данных эксперимента:
      user_id   gmv group_name
0  myo4ixol31  1428       test
1  myo4ixol31  1428       test
2  myo4ixol31  1071       test
3  myo4ixol31  1071       test
4  pkzf2889ww   351       test


3.2. Проверка гипотезы о равенстве средних (t-тест)

In [61]:
# Разделение на группы
control = user_gmv_exp[user_gmv_exp['group_name'] == 'control']['gmv']
test = user_gmv_exp[user_gmv_exp['group_name'] == 'test']['gmv']

# Проведение t-теста
t_stat, p_value = stats.ttest_ind(test, control, equal_var=False)

print("Результаты t-теста:")
print(f"T-статистика: {t_stat:.3f}")
print(f"P-value: {p_value:.3f}")

# Интерпретация результатов
alpha = 0.05
if p_value < alpha:
    print("Вывод: Отвергаем нулевую гипотезу - есть статистически значимые различия между группами.")
else:
    print("Вывод: Не отвергаем нулевую гипотезу - статистически значимых различий не обнаружено.")

Результаты t-теста:
T-статистика: 2.360
P-value: 0.018
Вывод: Отвергаем нулевую гипотезу - есть статистически значимые различия между группами.


## **4. Анализ среднего чека: дельта-метод**

4.1. Подготовка данных

In [62]:
# Агрегация данных с подсчетом количества транзакций
user_data = data_exp.groupby(['user_id', 'group_name']).agg(
    total_gmv=('gmv', 'sum'),
    n_transactions=('gmv', 'count')
).reset_index()

print("Агрегированные данные по пользователям:")
print(user_data.head())

Агрегированные данные по пользователям:
      user_id group_name  total_gmv  n_transactions
0  00062h7u56    control        733               1
1  00074uxybk       test       3187               3
2  000ic5j18m    control       2933               6
3  000plmykri       test       1695               5
4  00174ganru    control       1496               4


4.2. Реализация дельта-метода

In [63]:
def delta_method_variance(numerator, denominator):
    """
    Расчет дисперсии отношения средних (дельта-метод)

    Параметры:
    numerator - вектор числителя (например, сумма GMV)
    denominator - вектор знаменателя (например, количество транзакций)

    Возвращает:
    Оценку дисперсии отношения средних
    """
    x = numerator
    y = denominator
    n = len(x)

    mu_x = np.mean(x)
    mu_y = np.mean(y)
    var_x = np.var(x, ddof=1)
    var_y = np.var(y, ddof=1)
    cov_xy = np.cov(x, y, ddof=1)[0, 1]

    delta_var = (var_x / mu_y**2
                - 2 * cov_xy * mu_x / mu_y**3
                + var_y * mu_x**2 / mu_y**4) / n
    return delta_var

# Разделение на группы
control_group = user_data[user_data['group_name'] == 'control']
test_group = user_data[user_data['group_name'] == 'test']

# Расчет дисперсий по дельта-методу
test_var = delta_method_variance(test_group['total_gmv'], test_group['n_transactions'])
control_var = delta_method_variance(control_group['total_gmv'], control_group['n_transactions'])

# Расчет средних чеков
mean_control = control_group['total_gmv'].sum() / control_group['n_transactions'].sum()
mean_test = test_group['total_gmv'].sum() / test_group['n_transactions'].sum()

print("Результаты анализа среднего чека:")
print(f"Средний чек тестовой группы: {mean_test:.2f}")
print(f"Средний чек контрольной группы: {mean_control:.2f}")
print(f"Дисперсия тестовой группы (дельта-метод): {test_var:.4f}")
print(f"Дисперсия контрольной группы (дельта-метод): {control_var:.4f}")

Результаты анализа среднего чека:
Средний чек тестовой группы: 704.21
Средний чек контрольной группы: 700.22
Дисперсия тестовой группы (дельта-метод): 2.1727
Дисперсия контрольной группы (дельта-метод): 0.7142


4.3. Проверка гипотезы для среднего чека

In [65]:
# Расчет t-статистики и p-value
t_stat = (mean_test - mean_control) / np.sqrt(test_var + control_var)
df = len(test_group) + len(control_group) - 2  # степени свободы
p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df=df))

print("Результаты t-теста:")
print(f"T-статистика: {t_stat:.3f}")
print(f"P-value: {p_value:.3f}")

# Интерпретация результатов
alpha = 0.05
if p_value < alpha:
    print("Вывод: Отвергаем нулевую гипотезу - есть статистически значимые различия в среднем чеке между группами.")
else:
    print("Вывод: Не отвергаем нулевую гипотезу - статистически значимых различий в среднем чеке не обнаружено.")

Результаты t-теста:
T-статистика: 2.344
P-value: 0.019
Вывод: Отвергаем нулевую гипотезу - есть статистически значимые различия в среднем чеке между группами.


## **5. Линеаризация метрики**

5.1. Реализация линеаризации второго типа

In [66]:
def linearize_metric(x, y, x_bar, y_bar):
    """
    Линеаризация метрики отношения (второй тип)

    Параметры:
    x - числитель (total_gmv)
    y - знаменатель (n_transactions)
    x_bar - среднее по числителю
    y_bar - среднее по знаменателю

    Возвращает:
    Линеаризованные значения метрики
    """
    return x_bar/y_bar + (1/y_bar)*x - (x_bar/y_bar**2)*y

# Расчет средних значений для линеаризации
x_bar_control = control_group['total_gmv'].mean()
y_bar_control = control_group['n_transactions'].mean()
x_bar_test = test_group['total_gmv'].mean()
y_bar_test = test_group['n_transactions'].mean()

# Линеаризация метрик
control_linearized = linearize_metric(control_group['total_gmv'], control_group['n_transactions'], x_bar_control, y_bar_control)

test_linearized = linearize_metric(
    test_group['total_gmv'],
    test_group['n_transactions'],
    x_bar_test,
    y_bar_test
)

# Проведение t-теста на линеаризованных метриках
t_stat_linear, p_value_linear = stats.ttest_ind(test_linearized, control_linearized, equal_var=False)

print("Результаты анализа с линеаризацией метрики:")
print(f"T-статистика: {t_stat_linear:.3f}")
print(f"P-value: {p_value_linear:.3f}")

# Интерпретация результатов
if p_value_linear < alpha:
    print("Вывод: Отвергаем нулевую гипотезу - есть статистически значимые различия между группами.")
else:
    print("Вывод: Не отвергаем нулевую гипотезу - статистически значимых различий не обнаружено.")

Результаты анализа с линеаризацией метрики:
T-статистика: 2.344
P-value: 0.019
Вывод: Отвергаем нулевую гипотезу - есть статистически значимые различия между группами.


## **6. CUPED: повышение чувствительности теста**

6.1. Загрузка данных с историческими значениями

In [71]:
# Загрузка данных с историческими значениями
data_cuped = pd.read_csv('synthetic_gmv_data_1.3.csv')
print("Первые 5 строк данных для CUPED:")
print(data_cuped.head())

Первые 5 строк данных для CUPED:
   gmv_hist  gmv_exp group_name
0    200.78   123.19       test
1    363.80   134.49    control
2     39.93   116.72    control
3    150.99   177.67    control
4    208.93    65.30       test


6.2. Применение метода CUPED

In [72]:
# Расчет коэффициента theta
X = data_cuped['gmv_hist']
Y = data_cuped['gmv_exp']
theta = np.cov(X, Y)[0, 1] / np.var(X)
data_cuped['gmv_cuped'] = data_cuped['gmv_exp'] - theta * (data_cuped['gmv_hist'] - data_cuped['gmv_hist'].mean())

# Разделение на группы
test_cuped = data_cuped[data_cuped['group_name'] == 'test']
control_cuped = data_cuped[data_cuped['group_name'] == 'control']

6.3. Сравнение результатов до и после CUPED

In [75]:
# Проведение t-тестов
t_stat_orig, p_value_orig = stats.ttest_ind(test_cuped['gmv_exp'], control_cuped['gmv_exp'], equal_var=False)
t_stat_cuped, p_value_cuped = stats.ttest_ind(test_cuped['gmv_cuped'], control_cuped['gmv_cuped'], equal_var=False)

print("Сравнение результатов до и после применения CUPED:")
print(f"Оригинальные данные: p-value = {p_value_orig:.4f}")
print(f"CUPED p-value = {p_value_cuped:.4f}")


Сравнение результатов до и после применения CUPED:
Оригинальные данные: p-value = 0.2327
CUPED p-value = 0.0467
